
## 要解决的两个问题

1. 通过协变量估计真实的异质性效果
2. 对subsets上的效果进行统计推断

第一个挑战，大部分机器学习方法不能直接用于构建置信区间。因为它们是自适应的：通过训练数据来选择模型导致bias。在论文中通过”honesty“方法来解决。如果模型不通过同样的信息来选择模型和评估模型效果，那此模型就是”honesty“的。具体方法是，将样本一分为二，一半用于构建模型（包含交叉验证步骤），一半用于效果评估。这样做的坏处是由于样本量减少导致precision下降，好处是消除了bias

第二个挑战，训练模型需要基于可观测的”ground truth“，但是干预效果无法直接观测，论文中称为“fundamental problem of causal inference”。文中通过构建无偏的 mean-squared error 来解决。

## 1 问题
### 1.1 设定

假设有 $N$ 个单元，索引为 $i=1,...,N$。根据 Rubin 因果模型，每个单元都有潜在结果 $(Y_i(0), Y_i(1))$，每个单元的干预结果为 $\tau_i = Y_i(1) - Y_i(0)$，定义 $W_i$ 为单元 $i$ 是否进行干预，则：

$Y_i^{obs} = Y_i(W_i) =  \left\{\begin{matrix}
Y_i(0)\ \ \ if\ W_i = 0,
 \\
Y_i(1)\ \ \ if\ W_i = 1.
\end{matrix}\right. $    

如果 $X_i$ 是 $K$ 维度的特征向量，并且不受干预影响。    
则我们的数据集为 $(W_i, X_i, Y_i^{obs})$。定义被干预的概率为$p = pr(W_i = 1)$ ，$e(x)=pr(W_i | X_i = x)$为被干预的条件概率

### 1.2 无混杂
以下方法都假设控制了混杂，即：
$X \perp\perp  Y | X_i$    
 
### 1.3 条件干预效果和分区
定义条件干预效果（CATE）为：    $\tau(x) \equiv E[Y_i(1) - Y_i(0) | X_i = x]$    
论文中主要关于分区下的效果估计 $\tau( \cdot )$

## 2 整体均值的诚实估计

### 2.1 设定

$\Pi$ 是基于特征空间 $\mathbb{X}$ 划分的分区集合（树），$\#(\Pi)$ 是分区的个数：
$$\Pi = \{ \ell_1,  ..., \ell_{\#(\tau)} \},\; \ \ U_{j=1}^{\#(\Pi)}\ell_i = \mathbb{X}$$    

$\mathbb{P}$ 为分区空间。令$\ell(x,\Pi)$ 叶子节点 $\ell \in \Pi$，使得 $x \in \ell$。$\mathbb{S}$ 代表来自总体的数据样本空间。$\pi: \mathbb{S} \mapsto \mathbb{P}$ 代表通过样本 $S \in \mathbb{S}$ 构建分区的算法。    
对已知的分区，定义条件均值函数为：
$$\mu(x, \Pi) \equiv E[Y_i | X_i \in \ell(x, \Pi)] = E[\mu(X_i) | X_i \in \ell(x, \Pi)]$$
对已经的样本 $S$来说，其无偏估计为：
$$\hat{\mu}(x; S, \Pi) \equiv \frac{1} {\#(i \in S: X_i \in \ell(x, \Pi))} \sum_{i \in S: X_i \in \ell(x, \Pi)} Y_i$$


### 2.2 诚实的目标

论文中使用调整后的MSE
$$MSE(S^{te}, S^{est}, \Pi) \equiv \frac{1}{\#(S^{te})} \sum_{i \in S^{te}} \{(Y_i - \hat{\mu}(X_i; S^{est}, \Pi))^2 - Y_i^2\}$$    
其中 $S^{te}$ 是用于测试的样本，$S^{est}$ 是用于估计的样本。    其期望为：
$$EMSE(\Pi) \equiv \mathbb{E}_{S^{te}, S^{est}}[MSE(S^{te}, S^{est}, \Pi)]$$

论文最终目标是构造和评估最大化“诚实”标准的算法 $\pi(\cdot)$：
$$Q_{H}(\pi) \equiv -\mathbb{E}_{S^{te},S^{est}, S^{tr}}[MSE(S^(te),S^{est}, S^{tr})]$$

### 2.3 自适应的目标

在 CART 方法下会略有不同，即用同样的样本来构建树并评估效果：
$$Q_{C}(\pi) \equiv -\mathbb{E}_{S^{te},S^{tr}}[MSE(S^(te),S^{tr}, S^{tr})]$$

此类方法定义为”自适应的“，论文中新方法为”诚实的“

### 2.4 CART的实现方法
以$Q_{C}(\pi)$ 为优化目标，递归的构建树，通过交叉验证来选择树深度的惩罚来解决过拟合

### 2.5 诚实的划分
对 CART 来说有两方面修改：1. 通过独立样本 $S^{est}$ 来估计叶子的均值；2. 修改分流和交叉验证标准生成叶子的无偏估计，其中 $S^{est}$  在树构建阶段被视为随机变量。此外，明确地考虑了更精细的分区会导致叶估计的方差更大。    
$$-EMSE(\Pi) = -\mathbb{E}_{(Y_i, X_i), S^{est} }[(Y_i - \mu(X_i, \Pi)) ^ 2 - Y_i^2] - \mathbb{E}_{X_i, S^{est}}[(\hat{\mu}(X_i; S^{est}, \Pi) - \mu(X_i, \Pi)) ^ 2] 
\\ = \mathbb{E}_{X_i}[\mu^2(X_i;\Pi)] - \mathbb{E}_{S^{est},X_i}[\mathbb{V}(\hat{\mu}^2(X_i; S^{est}, \Pi))]$$

$$\hat{\mathbb{V}}(\hat{\mu}^2(X_i; S^{est}, \Pi)) \equiv \frac{S^2_{S^{tr}}(\ell(x;\Pi))}{N^{est}(\ell(x;\Pi))}$$
其中 $S^2_{S^{tr}}(\ell)$是叶子内的方差，可以如下近似：
$$\hat{\mathbb{E}}[\mathbb{V}(\hat{\mu}^2(X_i; S^{est}, \Pi)) | i \in S^{te}] 
\equiv \frac{1}{N^{est}} \sum_{\ell \in \Pi}S^2_{S^{tr}}(\ell)$$

评估方程中第一项为：
$$\hat{\mathbb{E}}\mu^2(x;\Pi) = \hat{\mu}^2(x, S^{tr}, \Pi) - \frac{S^2_{S^{st}}(\ell(x;\pi))}{N^{tr}(\ell(x;\pi))}$$

将上面的估计组合，并且如果实践中评估和训练的样本两相同，则最终：
$$\hat{EMSE}(S^{tr}, \Pi) \equiv \frac{1}{N^{tr}}\sum_{i \in S^{tr}} \hat{\mu}^2(X_i, S^{tr}, \Pi) 
- \frac{2}{N^{tr}}\sum_{\ell \in \Pi}S^2_{S^{tr}}(\ell)$$

对比 $MSE$ ，这里主要增加了方差项的调整

### 2.6 诚实的交叉验证

仅评估来自交叉验证样本 $S^{tr,cv}$ 的单元结果来考虑诚实估计：$-\hat{EMSE}(S^{tr,cv}, \Pi)$    

## 3 诚实的干预效果推断

将重点转移到估计条件平均治疗效果，而不是估计条件总体均值，本节估计量为“因果树”（Causal Tree, CT）估计量

$$MSE_\tau(S^{te}, S^{est}, \Pi) \equiv 
\frac{1}{\#(S^{te})} \sum\{(\tau_i - \hat{\tau}(X_i; S^{est}, \Pi))^2\ - \tau_i^2\}$$
$$EMSE_\tau(\Pi) \equiv \mathbb{E}_{S^{te}, S^{est}}[MSE_\tau(S^{te}, S^{est}, \Pi)]$$

然而我们无法观测 $\tau_i$

### 3.1 修改传统CART用于干预效果

$$-\hat{MSE}_\tau = \frac{1}{N^{tr}} \sum_{i \in S^{tr}} \hat{\tau}^2 (X_i; S^{tr}, \Pi)$$



### 3.2 改变为诚实的方法

$$-EMSE_\tau(\Pi) = \mathbb{E}_{X_i}[\tau^2(X_i;\Pi)] - \mathbb{E}_{S^{est},X_i}[\mathbb{V}(\hat{\tau}^2(X_i; S^{est}, \Pi))]$$

构建树只使用训练样本，这导致了不可行性标准的估计量仅取决于 $S^{tr}$：
$$\hat{EMSE}_\tau(S^{tr}, \Pi) \equiv \frac{1}{N^{tr}} \sum_{i \in S^{tr}} \hat{\tau}^2(X_i; S^{tr}, \Pi)
- \frac{2}{N^{tr}}\sum_{\ell \in \Pi}(\frac{S^2_{S^{tr}_{treat}}(\ell)}{p} + \frac{S^2_{S^{tr}_{control}}(\ell)}{p})$$

## 4 因果效应的四个分割估计量

### 4.1 Causal Trees (CT)
首选评估量

### 4.2 转化结果树 (TOT)

### 4.3 Fit-based Trees (F)

### 4.4 Squared T-statistic Trees (TS)